In [1]:
from appgeopy import *
from my_packages import *

In [2]:
gwl_h5 = "20240828_GWL_CRFP_model.h5"
gwl_data, gwl_metadata = h5pytools.open_HDF5(gwl_h5)
gwl_stations = [x for x in gwl_data.keys() if x != "date"]
date_arr = pd.to_datetime([x.decode("utf-8") for x in gwl_data["date"]])
len(gwl_stations), gwl_stations[:5]

(104, ['ANHE', 'ANNAN', 'BEIGANG', 'BOZI', 'CAICUO'])

In [16]:
# select_station = "YUANZHANG"

all_data = []

for select_station in tqdm(gwl_stations):
    try:
        gwldata_byStation = gwl_data[select_station]
        available_wells = sorted(gwldata_byStation.keys())

        active_wells = [
            well_code for well_code in available_wells if gwl_metadata[select_station][well_code]["Status"] == "Active"
        ]

        # select_well = active_wells[0]
        for select_well in active_wells:

            gwl_timeseries = pd.Series(data=gwldata_byStation[select_well]["measure"]["model"], index=date_arr)
            start_time = gwl_timeseries.first_valid_index()
            end_time = gwl_timeseries.last_valid_index()
            gwl_timeseries = gwl_timeseries.loc[start_time:end_time]

            sma_gwl_timeseries = pd.Series(
                smoothing.simple_moving_average(gwl_timeseries, window_size=31), index=gwl_timeseries.index
            )

            sma_gwl_timeseries = sma_gwl_timeseries[sma_gwl_timeseries.index.is_month_start]

            monthly_index_arr = [ele.strftime("%Y%m%d") for ele in sma_gwl_timeseries.index]
            monthly_value_arr = sma_gwl_timeseries.values

            data_byWell = {select_station: {select_well: {"measure":{"date": monthly_index_arr, "values": monthly_value_arr}}}}
            all_data.append(data_byWell)
    except Exception as e:
        print(select_station, e)
        pass

  0%|          | 0/104 [00:00<?, ?it/s]

GANYUAN 'Status'
TIANGZHONG 'Status'
XIABANTIAN 'Status'
XIGANG 'Status'
ZHONGLIAO 'Status'


In [17]:
monthly_gwl_data = merge_dicts(*all_data)
len(monthly_gwl_data.keys())

93

In [18]:
monthly_gwl_metadata = []

# add metadata based on the available stations in measurement data
for station in monthly_gwl_data.keys():
    available_wells = sorted(monthly_gwl_data[station].keys())
    for well in available_wells:
        well_metadata = gwl_metadata[station][well]
        monthly_gwl_metadata.append({station:{well:well_metadata}})

monthly_gwl_metadata = merge_dicts(*monthly_gwl_metadata)

monthly_gwl_metadata["CreatedTime"] = gwl_metadata["CreatedTime"]
monthly_gwl_metadata["Description"] = gwl_metadata["Description"]
monthly_gwl_metadata["Description_2"] = "2025/2/28 - Add monthly data of groundwater water level of activate stations"

In [19]:
today_string = datetime.now().strftime("%Y%m%d")

# Write updated data and metadata back to the HDF5 file
with h5py.File(f"{today_string}_Monthly_GWL.h5", "w") as hdf5_file:
    gwatertools.h5pytools.metadata_to_hdf5(hdf5_file, monthly_gwl_metadata)
    gwatertools.h5pytools.data_to_hdf5(hdf5_file, monthly_gwl_data)